<a href="https://colab.research.google.com/github/kangmg/rmsd_ipython/blob/master/rmsd_ipython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -q https://raw.githubusercontent.com/kangmg/rmsd_ipython/master/rmsd/calculate_rmsd.py -O rmsd_toolkit.py
!echo "Done!"

In [63]:
xyz1 = """6

Cl    0.000315113091   -4.851256508727    0.000000000000
Si    0.000008122666   -0.081479435428    0.000000000000
H   -1.409996918298   -0.481349820968    0.000000000000
H    0.705178765477   -0.481189816756   -1.221101893621
H    0.705178765477   -0.481189816756    1.221101893621
Br   -0.000147111879    2.196922164251    0.000000000000"""

xyz2 = """6

Cl    0.000308701706   -4.739114565961    0.000000000000
Si    0.000011888584   -0.119054586111    0.000000000000
H   -1.411266251779   -0.514082066748    0.000000000000
H    0.705205731485   -0.514578303328   -1.222045485239
H    0.705205731485   -0.514578303328    1.222045485239
Br   -0.000130084759    2.161823224660    0.000000000000"""

with open("xyz1.xyz", "w") as file:
  file.write(xyz1)

with open("xyz2.xyz", "w") as file:
  file.write(xyz2)

In [80]:
from rmsd_toolkit import *
from rmsd_toolkit import get_coordinates_xyz, centroid, check_reflections # utilities
from rmsd_toolkit import quaternion_rmsd, kabsch_rmsd, rmsd # rmsd
from rmsd_toolkit import reorder_hungarian, reorder_inertia_hungarian, reorder_brute, reorder_distance # reorder
import copy
import numpy as np

rmsd_options = lambda : print("""
_________________________________________________________________________________________\n""",
" \033[0;34mParameters\033[0m ","|","                            \033[0;34mSupported methods\033[0m\n"
"""____________|____________________________________________________________________________
  rotation  |                         'kabsch', 'quaternion'
____________|____________________________________________________________________________
  reorder   |            'hungarian', 'inertia-hungarian', 'brute', 'distance'
____________|____________________________________________________________________________

""", sep="")

def calculate_rmsd(P_str: str, Q_str: str, rotation_method: str = None, reorder_method: str = None, ignore_hydrogen: bool = False) -> float:
    '''

    '''

    P_all_atoms, P_all = get_coordinates_xyz(P_str, return_atoms_as_int=True)
    Q_all_atoms, Q_all = get_coordinates_xyz(Q_str, return_atoms_as_int=True)

    # Set local view
    p_view = None
    q_view = None

    if ignore_hydrogen:
        p_view = np.where(P_all_atoms != 1)
        q_view = np.where(Q_all_atoms != 1)

    # Set local view
    if p_view is None:
        P_coord = copy.deepcopy(P_all)
        Q_coord = copy.deepcopy(Q_all)
        P_atoms = copy.deepcopy(P_all_atoms)
        Q_atoms = copy.deepcopy(Q_all_atoms)

    else:
        P_coord = copy.deepcopy(P_all[p_view])
        Q_coord = copy.deepcopy(Q_all[q_view])
        P_atoms = copy.deepcopy(P_all_atoms[p_view])
        Q_atoms = copy.deepcopy(Q_all_atoms[q_view])

    P_size = P_atoms.shape[0]
    Q_size = Q_atoms.shape[0]

    # size check
    if not P_size == Q_size:
        raise ValueError(f"Structures not same size, first xyz : second xyz = {P_size} : {Q_size}")

    # Recenter to centroid
    P_cent = centroid(P_coord)
    Q_cent = centroid(Q_coord)
    P_coord -= P_cent
    Q_coord -= Q_cent

    # set rotation method
    if rotation_method == 'kabsch':
        rmsd_method = kabsch_rmsd
    elif rotation_method == 'quaternion':
        rmsd_method = quaternion_rmsd
    else:
        rmsd_method = rmsd

    # set reorder method
    if reorder_method == 'hungarian':
        reorder_method = reorder_hungarian
    elif reorder_method == 'inertia-hungarian':
        reorder_method = reorder_inertia_hungarian
    elif reorder_method == 'brute':
        reorder_method = reorder_brute
    elif reorder_method == 'distance':
        reorder_method = reorder_distance

    if reorder_method is not None:
        Q_review = reorder_method(P_atoms, Q_atoms, P_coord, Q_coord)
    else:
        Q_review = None

    # If there is a reorder, then apply before print
    if Q_review is not None:

        Q_atoms = Q_atoms[Q_review]
        Q_coord = Q_coord[Q_review]

        assert all(
            P_atoms == Q_atoms
        ), "error: Structure not aligned. Please submit bug report at http://github.com/charnley/rmsd"

    result_rmsd = rmsd_method(P_coord, Q_coord)

    return result_rmsd

In [73]:
AVAILABLE_ROTATION_METHODS = ['kabsch', 'quaternion', None]
AVAILABLE_REORDER_METHODS = [None, 'hungarian', 'inertia-hungarian', 'brute', 'distance']

In [81]:
import pandas as pd

df = pd.DataFrame(columns=["rotation_option", "reorder_option", "ignore_H", "RMSD"])

for rotation_option in AVAILABLE_ROTATION_METHODS:
  for reorder_option in AVAILABLE_REORDER_METHODS:
    for ignore_H in [True, False]:
      RMSD = calculate_rmsd("xyz1.xyz", "xyz2.xyz", rotation_method=rotation_option, reorder_method = reorder_option, ignore_hydrogen=ignore_H)
      new_row = pd.DataFrame([{"rotation_option": rotation_option, "reorder_option": reorder_option, "ignore_H": ignore_H, "RMSD": RMSD}], columns=["rotation_option", "reorder_option", "ignore_H", "RMSD"])
      df = pd.concat([df, new_row], ignore_index=True)

df

,rotation_option,reorder_option,ignore_H,RMSD
0,kabsch,None,True,0.070001
1,kabsch,None,False,0.054655
2,kabsch,hungarian,True,0.070001
3,kabsch,hungarian,False,0.054655
4,kabsch,inertia-hungarian,True,0.070001
5,kabsch,inertia-hungarian,False,0.054655
6,kabsch,brute,True,0.070001
7,kabsch,brute,False,0.054655
8,kabsch,distance,True,0.070001
9,kabsch,distance,False,0.054655


In [75]:
rmsd_options()


_________________________________________________________________________________________
 Parameters |                            Supported methods
____________|____________________________________________________________________________
  rotation  |                         'kabsch', 'quaternion'
____________|____________________________________________________________________________
  reorder   |            'hungarian', 'inertia-hungarian', 'brute', 'distance'
____________|____________________________________________________________________________




In [76]:
rmsd_value = calculate_rmsd("xyz1.xyz", "xyz2.xyz", reorder_method = "inertia-hungarian", rotation_method=None)
print(rmsd_value)
!python3 rmsd_toolkit.py --reorder --reorder-method  inertia-hungarian  --rotation none xyz1.xyz xyz2.xyz

1.7284287314020776
1.7284287314020776


In [77]:
rmsd_value = calculate_rmsd("xyz1.xyz", "xyz2.xyz", reorder_method = "inertia-hungarian", rotation_method='kabsch', ignore_hydrogen=True)
print(rmsd_value)
!python3 rmsd_toolkit.py --reorder --reorder-method  inertia-hungarian  --rotation kabsch  --ignore-hydrogen xyz1.xyz xyz2.xyz

0.07000096634081729
0.07000096634081729


In [78]:
rmsd_value = calculate_rmsd("xyz1.xyz", "xyz2.xyz", reorder_method = "inertia-hungarian", rotation_method='kabsch')
print(rmsd_value)
!python3 rmsd_toolkit.py --reorder --reorder-method  inertia-hungarian  --rotation kabsch xyz1.xyz xyz2.xyz

0.05465458928596824
0.05465458928596824
